In [ ]:
import weaviate
from Supressout import suppress_output


In [14]:
pip install weaviate-client


  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached validators-0.34.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached validators-0.34.0-py3-none-any.whl (43 kB)
Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 13.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
from fastembed import TextEmbedding


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
COLLECTION_NAME = "Books"  # capitalize the first letter of collection names
BOOK_DESCRIPTION_FOLDER = "include/data"
EMBEDDING_MODEL_NAME = "BAAI/bge-small-en-v1.5"

In [ ]:

#instantiate local weaviate client
with suppress_output():
    client = weaviate.connect_to_local(
       
    )
print("Started new embedded Weaviate instance.")
print(f"Client is ready: {client.is_ready()}")

Started new embedded Weaviate instance.
Client is ready: True


In [16]:
#creating the collection
existing_collections= client.collections.list_all()
existing_collection_names=existing_collections.keys()

if COLLECTION_NAME not in existing_collection_names:
    print(f"Collections{COLLECTION_NAME}does not exist yet.Creating it...")
    collection=client.collections.create(name=COLLECTION_NAME)
    print(f"Collection {COLLECTION_NAME} created successfully.")
else:
    print(f"Collection{COLLECTION_NAME} already exists")
    collection= client.collections.get(COLLECTION_NAME)

CollectionBooks already exists


In [20]:
#Extract text from local files
#list the book description files
import os
book_description_files = [
    f for f in os.listdir(BOOK_DESCRIPTION_FOLDER)
    if f.endswith('.txt')
    
]
print(f"The following files with book descriptions were found: {book_description_files}")

The following files with book descriptions were found: ['my_book_descriptions.txt']


In [21]:
# Add your own book description file
# Format 
# [Integer Index] ::: [Book Title] ([Release year]) ::: [Author] ::: [Description]

my_book_description = """0 ::: The Idea of the World (2019) ::: Bernardo Kastrup ::: An ontological thesis arguing for the primacy of mind over matter.
1 ::: Exploring the World of Lucid Dreaming (1990) ::: Stephen LaBerge ::: A practical guide to learning and enjoying lucid dreams.
2 ::: Don’t Let Him In (2025) ::: Lisa Jewell ::: A gripping psychological thriller.
3 ::: Atmosphere (2025) ::: Taylor Jenkins Reid ::: Contemporary fiction exploring identity and relationships.
4 ::: A Mother’s Love (2025) ::: Danielle Steel ::: Emotional family drama.
5 ::: Never Flinch (2025) ::: Stephen King ::: A chilling thriller.
6 ::: The Knight and the Moth (2025) ::: Rachel Gillig ::: Epic fantasy about destiny and rebellion.
7 ::: Great Big Beautiful Life (2025) ::: Emily Henry ::: Heart-warming contemporary fiction.
8 ::: My Friends (2025) ::: Fredrik Backman ::: Reflective literary fiction.
9 ::: Original Sin (2025) ::: Tapper & Thompson ::: Political non-fiction exploring truth and power.
10 ::: The Let Them Theory (2025) ::: Mel Robbins ::: Motivational self-help on boundaries and mindset.
11 ::: Baking Across America (2025) ::: B. Dylan Hollis ::: Food memoir and cookbook.
12 ::: Iron Flame (2024) ::: Rebecca Yarros ::: High-fantasy sequel with dragons and rebellion.
13 ::: Fourth Wing (2023) ::: Rebecca Yarros ::: Fantasy romance bestseller.
14 ::: Onyx Storm (2024) ::: Rebecca Yarros ::: Romantic fantasy adventure.
15 ::: One Golden Summer (2025) ::: Carley Fortune ::: Contemporary romance about second chances.
16 ::: The Tenant (2025) ::: Freida McFadden ::: Psychological thriller full of suspense.
17 ::: The Boyfriend (2025) ::: Freida McFadden ::: Twisty and unsettling domestic thriller.
18 ::: The Crash (2025) ::: Freida McFadden ::: Gripping thriller involving memory and mystery.
19 ::: Jujutsu Kaisen Vol. 26 (2025) ::: Gege Akutami ::: Popular shōnen manga with supernatural battles.
20 ::: Solo Leveling Vol. 12 (2025) ::: Dubu & Chugong ::: Hit fantasy webtoon adaptation.
21 ::: The Great Gatsby (1925) ::: F. Scott Fitzgerald ::: Classic American novel of wealth and illusion.
22 ::: Atlas of the Heart (2021) ::: Brené Brown ::: Guide to emotional language and connection.
23 ::: The Four Winds (2021) ::: Kristin Hannah ::: Historical fiction during the Great Depression.
24 ::: How Not to Die (2015) ::: Michael Greger, MD ::: Health and nutrition science-based guide.
25 ::: And Then There Were None (1939) ::: Agatha Christie ::: Masterful locked-room mystery.
26 ::: The Da Vinci Code (2003) ::: Dan Brown ::: Worldwide thriller about secret societies.
27 ::: She: A History of Adventure (1887) ::: H. Rider Haggard ::: Adventure fantasy novel.
28 ::: Harry Potter and the Chamber of Secrets (1998) ::: J.K. Rowling ::: Second book in the iconic fantasy series.
29 ::: Harry Potter and the Deathly Hallows (2007) ::: J.K. Rowling ::: Final epic in the Harry Potter saga.
30 ::: The Girl with the Dragon Tattoo (2005) ::: Stieg Larsson ::: International crime thriller.
31 ::: Where the Crawdads Sing (2018) ::: Delia Owens ::: Literary mystery about loneliness and survival.
32 ::: 1984 (1949) ::: George Orwell ::: Dystopian vision of a totalitarian future.
33 ::: The 7 Habits of Highly Effective People (1989) ::: Stephen R. Covey ::: Classic personal development guide.
34 ::: To Kill a Mockingbird (1960) ::: Harper Lee ::: Pulitzer-winning novel on race and justice.
35 ::: The Alchemist (1988) ::: Paulo Coelho ::: Spiritual journey of personal legend.
36 ::: The Hobbit (1937) ::: J.R.R. Tolkien ::: Fantasy adventure and prelude to The Lord of the Rings.
37 ::: A Game of Thrones (1996) ::: George R.R. Martin ::: Political intrigue in a medieval fantasy world.
38 ::: The Very Hungry Caterpillar (1969) ::: Eric Carle ::: Beloved children’s picture book.
39 ::: Think and Grow Rich (1937) ::: Napoleon Hill ::: Foundational self-help and business mindset book.
40 ::: Anne of Green Gables (1908) ::: L.M. Montgomery ::: Classic coming-of-age story.
41 ::: Black Beauty (1877) ::: Anna Sewell ::: Moving story told from a horse’s perspective.
42 ::: The Name of the Rose (1980) ::: Umberto Eco ::: Medieval mystery and philosophical novel.
43 ::: Charlotte’s Web (1952) ::: E.B. White ::: Heartwarming tale of friendship and loss.
44 ::: The Eagle Has Landed (1975) ::: Jack Higgins ::: WWII thriller with suspense and deception.
45 ::: Sophie’s World (1991) ::: Jostein Gaarder ::: Philosophical novel exploring the history of thought.
46 ::: Dune (1965) ::: Frank Herbert ::: Epic science fiction saga on politics and ecology.
47 ::: Man’s Search for Meaning (1946) ::: Viktor E. Frankl ::: Memoir and psychology of survival and purpose.
48 ::: Catch-22 (1961) ::: Joseph Heller ::: Satirical anti-war novel with paradoxical logic.
49 ::: The Catcher in the Rye (1951) ::: J.D. Salinger ::: Classic coming-of-age story of teenage alienation."""

# Write to file
with open(f"{BOOK_DESCRIPTION_FOLDER}/my_book_descriptions.txt", 'w') as f:
    f.write(my_book_description)


In [22]:
book_description_files = [
    f for f in os.listdir(BOOK_DESCRIPTION_FOLDER)
    if f.endswith('.txt')
]

list_of_book_data = []

for book_description_file in book_description_files:
    with open(
        os.path.join(BOOK_DESCRIPTION_FOLDER, book_description_file), "r"
    ) as f:
        book_descriptions = f.readlines()
    
    titles = [
        book_description.split(":::")[1].strip()
        for book_description in book_descriptions
    ]
    authors = [
        book_description.split(":::")[2].strip()
        for book_description in book_descriptions
    ]
    book_description_text = [
        book_description.split(":::")[3].strip()
        for book_description in book_descriptions
    ]
    
    book_descriptions = [
        {
            "title": title,
            "author": author,
            "description": description,
        }
        for title, author, description in zip(
            titles, authors, book_description_text
        )
    ]

    list_of_book_data.append(book_descriptions)

In [23]:
import json
from IPython.display import JSON
JSON(json.dumps(list_of_book_data))

/Users/shreyabaral/Library/Python/3.11/lib/python/site-packages/IPython/core/display.py:661: UserWarning: JSON expects JSONable dict or list, not JSON strings
  warnings.warn("JSON expects JSONable dict or list, not JSON strings")


<IPython.core.display.JSON object>

In [24]:
#create vector embeddings from descriptions 
embedding_model= TextEmbedding(EMBEDDING_MODEL_NAME)
list_of_description_embeddings = []
for book_data in list_of_book_data:
    book_description= [book['description'] for book in book_data]
    description_embeddings= [
    list(embedding_model.embed([desc]))[0]for desc in book_description]
    list_of_description_embeddings.append(description_embeddings)

In [33]:
from weaviate.classes.data import DataObject

#load embeddings to weaviate
for book_data_list,emb_list in zip(list_of_book_data,list_of_description_embeddings):
    items=[]

    for book_data,emb in zip(book_data_list,emb_list):
        item=DataObject(
            properties={
                "title":book_data["title"],
                "author":book_data["author"],
                "description":book_data["description"],
              
            },
            vector=emb,
            
        )
        items.append(item)
    collection.data.insert_many(items)
    

In [34]:
query_str = "A philosophical book"

embedding_model = TextEmbedding(EMBEDDING_MODEL_NAME)  
collection = client.collections.get(COLLECTION_NAME)

query_emb = list(embedding_model.embed([query_str]))[0]

results = collection.query.near_vector(
    near_vector=query_emb,
    limit=1,
)
for result in results.objects:
    print(f"You should read: {result.properties['title']} by {result.properties['author']}")
    print("Description:")
    print(result.properties["description"])

You should read: Sophie’s World (1991) by Jostein Gaarder
Description:
Philosophical novel exploring the history of thought.
